# MNIST Image Classification using CNN inserted with Inception Modules in Pytorch - 
### Toby Moreno

Traditional ANN and classic machine learning classifiers could only push the accuracy of correctly predicting MNIST images to around 97-98%.  However, with just 12 epochs and the similar hyper-parametrs, Convolutional Neueral Network (CNN) constructed with inception modules, could overcome that limit with an accuracy closely breaking the __99%__ accuracy.  Test set: Average loss: 0.0415, Accuracy: [9876/10000, 9888/10000]

## CNN Architecture

CNNs use a variation of multilayer perceptrons designed to require minimal preprocessing.[1] They are also known as shift invariant or space invariant artificial neural networks (SIANN), based on their shared-weights architecture and translation invariance characteristics ...

CNNs use relatively little pre-processing compared to other image classification algorithms. This means that the network learns the filters that in traditional algorithms were hand-engineered. This independence from prior knowledge and human effort in feature design is a major advantage.- Wikipedia


![](http://parse.ele.tue.nl/cluster/2/CNNArchitecture.jpg)

Credit Source:


> Unlike traditional Neural Networks where all layers are fully-meshed, CNNs are much efficient because it uses a variation of partially-meshed multilayer perceptrons designed to require minimal preprocessing. Also the manual, time-consuming, and very subjective feature engineering of the images typically associated in traditional NN designs doesn’t apply to CNN because they are built-in the training process.  And finally, CNN’s convolved features of low-dimension matrices, the by-product output weights of the middle layers, are spatial invariant.  This is a significant because entities, texts, or objects could be tagged similar regardless of size, orientation, and spatial position in the image.

In [29]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

## Download MNIST images

For the image dataset, Pytorch has a toolkit to download MNIST data for training and testing.  Inside the package torchvision, import datasets and set the download parameter to True.  It has a built-in mechanism to transform also the data and target to tensor data structures

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)


test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())


![](https://github.com/vdumoulin/conv_arithmetic)


![](https://tobymoreno.github.io/images/cnn_mnist.png)


CNN network architectures are commonly deep (see above). But deep networks are prone to overfitting difficult to pass gradient updates through the entire network.  Additionally, just stacking large convolution operations is computationally expensive.  Researches in Google and South Carolina came up with a wider network where filters with multiple sizes operate on the same level.  For example, the below image is the “naive” inception module. It performs convolution on an input, with 3 different sizes of filters (1x1, 3x3, 5x5). Additionally, max pooling is also performed. The outputs are concatenated and sent to the next inception module.

Paper: https://arxiv.org/pdf/1409.4842v1.pdf

## Modified CNN with inception Modules 

![](https://tobymoreno.github.io/images/inception_modules.001.png)

Source: https://www.youtube.com/watch?v=VxhSouuSZDY

Let's look at the sample data in the image

In [3]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./mnist_data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [4]:
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: ./mnist_data/
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [5]:
ls

Classify Code Base-Copy1.ipynb         Test2.ipynb
Classify Code Base.ipynb               code_analysis-Copy1.ipynb
Image Classification using CNN.ipynb   code_analysis.ipynb
MNIST Pytorch.ipynb                    data/
Pytorch Machine Learning Primer.ipynb  file type classification.ipynb
Softmax Loss.ipynb                     mnist_data/
Test.ipynb


## DataLoader

DataLoader is pytorch's framework to standardized and scale up the training and testing process.

As long as your dataset conforms to a certain standard interface.. it could enumerate and perform automatic shuffling to fetch the next record in the dataset

You shall implement the following interface:

__getitem__ and __len__



In [6]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Basic CNN Module without Inception

In [7]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

#### Network Summary for Inception (CNN)

```
Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(5, 5), stride=(1, 1))
  (incept1): InceptionA(
    (branch1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3dbl_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3dbl_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3dbl_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch_pool): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
  )
  (incept2): InceptionA(
    (branch1x1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3dbl_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3dbl_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3dbl_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch_pool): Conv2d(20, 24, kernel_size=(1, 1), stride=(1, 1))
  )
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1408, out_features=10, bias=True)
)
```


In [27]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py

class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


![](https://tobymoreno.github.io/images/codemap.png)

Source: https://docs.google.com/presentation/d/1MJxGye-VZVoQfN1yFJlwwgcXxhn0jWB19ZxYYg8iKxY/edit#slide=id.g2901a5d332_174_275

In [28]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [26]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Run code below couple of times: 4 to 5
```
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.031828
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.005347
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.006505
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.003654
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.006652

Test set: Average loss: 0.0413, Accuracy: 9888/10000 (98.88%)
```

In [23]:
for epoch in range(1, 10):
    train(epoch)
    test()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:87: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.015831
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.004627
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.004567
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.000091
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.007195
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.030553
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.017647
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.002286
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.072995
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.010806
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.003382
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.002586
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.041820
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.003996
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.000646
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.021575
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.051365
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.025359
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.005306
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.078589
Train Epoch: 1 [12800/60000 (

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0426, Accuracy: 9877/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.013732
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.004916
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.003251
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.038598
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.024728
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.018196
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.027254
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.012725
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.026995
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.010221
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.037582
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.003763
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.005803
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.113117
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.011915
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.006110
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.046443
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.002394
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.061287
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.005072
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.013483
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.003587
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.003143
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.005576
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.001818
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.041485
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.029820
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.008391
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.007983
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.045594
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.002832
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.018932
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.070114
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.015018
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.020849
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.025225
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.023740
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.010051
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.015681


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.037473
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.004423
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.003373
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.098624
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.026053
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.002063
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.039637
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.004274
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.051788
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.031249
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.008972
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.054904
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.031542
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.001057
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.036127
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.003358
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.043570
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.008284
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.021081
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.027123


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.009544
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.001391
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.010577
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.002078
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.000450
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.007443
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.017045
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.009549
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.019111
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.006644
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.006813
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.047441
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.016003
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.033330
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.058550
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.000079
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.028814
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.000581
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.014159
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.012409


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.031828
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.005347
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.006505
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.003654
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.006652

Test set: Average loss: 0.0413, Accuracy: 9888/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.011973
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.009975
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.015808
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.038299
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.006253
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.005927
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.009949
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.001249
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.009757
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.025707
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.030619
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.012213
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.002982
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.014038
Train Epoch: 9 [